In [81]:
%pip install langchain_community python-magic-bin tiktoken langchain-openai libmagic langchainhub chromadb langchain unstructured

In [82]:
from langchain_openai import ChatOpenAI
import os
os.environ["LANGSMITH_TRACING"] = "True"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = ""
os.environ["OPENAI_API_KEY"] = ""

---

In [83]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_openai import ChatOpenAI
from pathlib import Path


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20200337\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\20200337\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\20200337\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [84]:
data_dir = r'data'
data_dir

'data'

In [85]:
loader = DirectoryLoader(
        str(data_dir),
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader,
        show_progress=True,
        use_multithreading=True
    )
loader

In [86]:
documents = loader.load()
documents

100%|██████████| 3/3 [00:00<00:00,  8.72it/s]


[Document(metadata={'source': 'data\\bedrock_or_sagemaker.md'}, page_content="AWS Decision guide\n\nAmazon Bedrock or Amazon SageMaker AI?\n\nCopyright © 2024 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.\n\nAmazon Bedrock or Amazon SageMaker AI?: AWS Decision guide\n\nCopyright © 2024 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.\n\nAmazon's trademarks and trade dress may not be used in connection with any product or service that is not Amazon's, in any manner that is likely to cause confusion among customers, or in any manner that disparages or discredits Amazon. All other trademarks not owned by Amazon are the property of their respective owners, who may or may not be affiliated with, connected to, or sponsored by Amazon.\n\nTable of Contents\n\nDecision guide.................................................................................................................................. 1 Introduction..................................

In [87]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)

chuncks = text_splitter.split_documents(documents)

In [88]:
len(chuncks), chuncks[0]

(143,
 Document(metadata={'source': 'data\\bedrock_or_sagemaker.md', 'start_index': 0}, page_content="AWS Decision guide\n\nAmazon Bedrock or Amazon SageMaker AI?\n\nCopyright © 2024 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.\n\nAmazon Bedrock or Amazon SageMaker AI?: AWS Decision guide\n\nCopyright © 2024 Amazon Web Services, Inc. and/or its affiliates. All rights reserved.\n\nAmazon's trademarks and trade dress may not be used in connection with any product or service that is not Amazon's, in any manner that is likely to cause confusion among customers, or in any manner that disparages or discredits Amazon. All other trademarks not owned by Amazon are the property of their respective owners, who may or may not be affiliated with, connected to, or sponsored by Amazon.\n\nTable of Contents"))

---

# Chroma DB

In [89]:
CHROMA_DB_PATH = Path("chroma_db")

In [90]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

db = Chroma.from_documents(
    chuncks, OpenAIEmbeddings(), persist_directory=str(CHROMA_DB_PATH))

In [91]:
db

---

# Query the db

In [92]:
embedding_function = OpenAIEmbeddings()

In [93]:
db  = Chroma(persist_directory=str(CHROMA_DB_PATH),
             embedding_function=embedding_function)      

In [110]:
query_text = """How to use AWS Batch to run batch computing workloads in the AWS Cloud."""

In [111]:
results  = db.similarity_search_with_relevance_scores(query_text, k=3)

In [112]:
results

[(Document(metadata={'source': 'data\\containers-on-aws-how-to-choose.md', 'start_index': 21247}, page_content='Explore the guide Amazon Lightsail resource center Explore Lightsail tutorials, videos, and links to core concept documentation. Visit the resource center AWS Batch\n\nWhat is AWS Batch? Learn how to use AWS Batch to run batch computing workloads in the AWS Cloud. Explore the guide Best practices for AWS Batch Consider this guidance on how to run and optimize your workloads when using AWS Batch. Explore the guide AWS Batch workshops center Vertical solutions 15\n\nUse these workshops, organized in a progressive manner from beginner to advanced, to explore and learn AWS Batch. Explore the workshops\n\nTools and services with container support......................................................................................\n\nAWS Copilot'),
  0.8591639770874674),
 (Document(metadata={'source': 'data\\containers-on-aws-how-to-choose.md', 'start_index': 21247}, page_content=

In [113]:
for content in results:
    print(content[0].page_content)

Explore the guide Amazon Lightsail resource center Explore Lightsail tutorials, videos, and links to core concept documentation. Visit the resource center AWS Batch

What is AWS Batch? Learn how to use AWS Batch to run batch computing workloads in the AWS Cloud. Explore the guide Best practices for AWS Batch Consider this guidance on how to run and optimize your workloads when using AWS Batch. Explore the guide AWS Batch workshops center Vertical solutions 15

Use these workshops, organized in a progressive manner from beginner to advanced, to explore and learn AWS Batch. Explore the workshops

Tools and services with container support......................................................................................

AWS Copilot
Explore the guide Amazon Lightsail resource center Explore Lightsail tutorials, videos, and links to core concept documentation. Visit the resource center AWS Batch

What is AWS Batch? Learn how to use AWS Batch to run batch computing workloads in the AWS C

In [114]:
if len(results) == 0 or results[0][1] < 0.7:
    print("No relevant documents found")

In [115]:
context = results[0][0].page_content

In [116]:
context

'Explore the guide Amazon Lightsail resource center Explore Lightsail tutorials, videos, and links to core concept documentation. Visit the resource center AWS Batch\n\nWhat is AWS Batch? Learn how to use AWS Batch to run batch computing workloads in the AWS Cloud. Explore the guide Best practices for AWS Batch Consider this guidance on how to run and optimize your workloads when using AWS Batch. Explore the guide AWS Batch workshops center Vertical solutions 15\n\nUse these workshops, organized in a progressive manner from beginner to advanced, to explore and learn AWS Batch. Explore the workshops\n\nTools and services with container support......................................................................................\n\nAWS Copilot'

---

In [117]:
prompt_template = f"""Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {query_text}
"""

In [118]:
prompt_template

'Answer the question based only on the following context:\n\nExplore the guide Amazon Lightsail resource center Explore Lightsail tutorials, videos, and links to core concept documentation. Visit the resource center AWS Batch\n\nWhat is AWS Batch? Learn how to use AWS Batch to run batch computing workloads in the AWS Cloud. Explore the guide Best practices for AWS Batch Consider this guidance on how to run and optimize your workloads when using AWS Batch. Explore the guide AWS Batch workshops center Vertical solutions 15\n\nUse these workshops, organized in a progressive manner from beginner to advanced, to explore and learn AWS Batch. Explore the workshops\n\nTools and services with container support......................................................................................\n\nAWS Copilot\n\n---\n\nAnswer the question based on the above context: How to use AWS Batch to run batch computing workloads in the AWS Cloud.\n'

In [119]:
prompt = prompt_template

In [120]:
model = ChatOpenAI()
response_text = model.predict(prompt)

In [121]:
sources = [doc.metadata.get("source", None) for doc, _score in results]
sources

['data\\containers-on-aws-how-to-choose.md',
 'data\\containers-on-aws-how-to-choose.md',
 'data\\containers-on-aws-how-to-choose.md']

In [122]:
formatted_response = f"Response: {response_text}\nSources: {sources}"
formatted_response

"Response: To use AWS Batch to run batch computing workloads in the AWS Cloud, you can explore the guide and best practices provided in the AWS Batch resource center. Additionally, you can participate in workshops to gain hands-on experience and learn more about utilizing AWS Batch effectively.\nSources: ['data\\\\containers-on-aws-how-to-choose.md', 'data\\\\containers-on-aws-how-to-choose.md', 'data\\\\containers-on-aws-how-to-choose.md']"